In [1]:
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
calls = pd.read_csv(snakemake.input[0], sep="\t")
# calls = pd.read_csv("/local/thomas/master-snakemake-workflow-sars-cov2/results/benchmarking/tables/aggregated_read_calls.tsv", sep="\t")
ordered_state = {
    "read" : "1. Kallisto",
    "contig" : "2. Pangoling/Contig",
    "scaffold" : "3. Pangolin/Scaffold",
    "polished_scaffold" : "4. Pangolin/Polished Scaffold",
    "pseudo" : "5. Pangolin/Pseudo"
}
calls["state"].replace(ordered_state, inplace=True)
calls

,lineage,actual_lineage,read_number,read_length,correct_call,state,sample
0,None,AY.6,100,100,False,2. Contig,0
1,None,AY.6,100,100,False,3. Scaffold,0
2,None,AY.6,100,100,False,4. Pol. Scaffold,0
3,None,AY.6,100,100,False,5. Pseudo,0
4,AY.6,AY.6,100,100,True,1. Read,0
...,...,...,...,...,...,...,...
14995,B.1.249,B.1.249,1000000,250,True,2. Contig,2999
14996,B.1.249,B.1.249,1000000,250,True,3. Scaffold,2999
14997,B.1.249,B.1.249,1000000,250,True,4. Pol. Scaffold,2999
14998,B.1.249,B.1.249,1000000,250,True,5. Pseudo,2999


In [ ]:
sort_order = ['read', 'contig', 'scaffold', 'polished_scaffold', 'pseudo']

heatmap = alt.Chart(calls).mark_bar().encode(
    y=alt.Y('read_number:O', axis=alt.Axis(format=',.0f'), title='Read Count per Sample'),
    x=alt.X('read_length:O', title='Read Length'),
    color=alt.Color('sum(correct_call):Q', title='Lineage correctly called', legend=alt.Legend(orient='right')),    
    tooltip=['sum(correct_call)', 'count(correct_call)'],
    text='sum(correct_call):Q'
)

text = alt.Chart().mark_text(baseline='middle', fontSize=7).encode(
    y='read_number:O',
    x='read_length:O',
    text='sum(correct_call):Q',
    color=alt.value('white'),
)

# Draw the chart.
alt.layer(heatmap, text, data=calls).facet(
    facet=alt.Facet('state', title='Lineage called on'),
).save(snakemake.output[0])
